In [1]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [2]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

C:\Users\user\Documents\GitHub\2nd-DL-CVMarathon\homework\Object Detection 程式導讀\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [3]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [4]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [5]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [6]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [7]:
PriorBox_Demo=PriorBox(cfg)

In [8]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [9]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [10]:
Use_cuda=False
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [11]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

In [12]:
'''要不要使用gpu'''
Use_cuda=False

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

Run with --cuda for optimal training speed.


---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [13]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

C:\Users\user\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
epochs=300
iteration=1000

In [ ]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'weights/Ｗeights.pth')

C:\Users\user\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\user\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
C:\Users\user\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BBOX Regression Loss:  2.509421624077691
Classification Loss:  14.592314430519389
BBOX Regression Loss:  2.5092891269259985
Classification Loss:  14.507457422327112
BBOX Regression Loss:  2.5091568369924278
Classification Loss:  14.427870460792825
BBOX Regression Loss:  2.50902468363444
Classification Loss:  14.348880061396844
BBOX Regression Loss:  2.5088925340440538
Classification Loss:  14.26992149070457
BBOX Regression Loss:  2.5087600896387925
Classification Loss:  14.190849869339557
BBOX Regression Loss:  2.508627295115638
Classification Loss:  14.111491628172537
BBOX Regression Loss:  2.508494320622197
Classification Loss:  14.031802156236438
BBOX Regression Loss:  2.5083612732435943
Classification Loss:  13.951912183722351
BBOX Regression Loss:  2.508228101377134
Classification Loss:  13.871865143952546
BBOX Regression Loss:  2.5080948081482135
Classification Loss:  13.791765155214256
BBOX Regression Loss:  2.507961423897449
Classification Loss:  13.711927710639108
BBOX Regress

BBOX Regression Loss:  2.4791160300925927
Classification Loss:  12.137665360062211
BBOX Regression Loss:  2.478819500958478
Classification Loss:  12.137169110333478
BBOX Regression Loss:  2.478515738028067
Classification Loss:  12.136664270471641
BBOX Regression Loss:  2.478203957169144
Classification Loss:  12.136150783962673
BBOX Regression Loss:  2.4778838150589553
Classification Loss:  12.135626650209781
BBOX Regression Loss:  2.477555880134488
Classification Loss:  12.13509253984616
BBOX Regression Loss:  2.4772201578453106
Classification Loss:  12.13454812670511
BBOX Regression Loss:  2.4768762765107333
Classification Loss:  12.133993092289677
BBOX Regression Loss:  2.4765235084565087
Classification Loss:  12.133427648976017
BBOX Regression Loss:  2.4761611147280096
Classification Loss:  12.132851856372977
BBOX Regression Loss:  2.4757882448960635
Classification Loss:  12.13226540305398
BBOX Regression Loss:  2.4754042637201006
Classification Loss:  12.131666875768591
BBOX Regres

BBOX Regression Loss:  1.858448890403465
Classification Loss:  10.604325471100982
BBOX Regression Loss:  1.855393558078342
Classification Loss:  10.591002683286314
BBOX Regression Loss:  1.852307609275535
Classification Loss:  10.577529153706115
BBOX Regression Loss:  1.8491913089045773
Classification Loss:  10.563859614619503
BBOX Regression Loss:  1.8460729782669634
Classification Loss:  10.549963017216434
BBOX Regression Loss:  1.8429456145675094
Classification Loss:  10.535821024576824
BBOX Regression Loss:  1.8398047633902734
Classification Loss:  10.521421240751074
BBOX Regression Loss:  1.8366453718256068
Classification Loss:  10.506754246464483
BBOX Regression Loss:  1.8334637426054525
Classification Loss:  10.491791499887475
BBOX Regression Loss:  1.8302705227887186
Classification Loss:  10.476493100766783
BBOX Regression Loss:  1.8270624244253244
Classification Loss:  10.460843347620083
BBOX Regression Loss:  1.823833426722774
Classification Loss:  10.444822871832201
BBOX Reg

BBOX Regression Loss:  0.4251615771540889
Classification Loss:  3.801601381655092
BBOX Regression Loss:  0.41722421292905454
Classification Loss:  3.7840268170392073
BBOX Regression Loss:  0.4097364531622993
Classification Loss:  3.7663848688573007
BBOX Regression Loss:  0.40268203947279185
Classification Loss:  3.7487834223994505
BBOX Regression Loss:  0.39597010365238894
Classification Loss:  3.7312833376284
BBOX Regression Loss:  0.3895044262026563
Classification Loss:  3.714011924649463
BBOX Regression Loss:  0.38328691411901405
Classification Loss:  3.6969879554062297
BBOX Regression Loss:  0.37733756171332467
Classification Loss:  3.680121114518908
BBOX Regression Loss:  0.3715935298935377
Classification Loss:  3.6633837130825215
BBOX Regression Loss:  0.36604525000960747
Classification Loss:  3.646746557730216
BBOX Regression Loss:  0.36070038124367043
Classification Loss:  3.6301731122463248
BBOX Regression Loss:  0.35554412014690445
Classification Loss:  3.613646832218877
BBOX

...

BBOX Regression Loss:  0.0020861618056162226
Classification Loss:  0.7820099431897629
BBOX Regression Loss:  0.0020692321330376493
Classification Loss:  0.7814376190946756
BBOX Regression Loss:  0.00205217192546676
Classification Loss:  0.7808744035942744
BBOX Regression Loss:  0.0020352301217132695
Classification Loss:  0.7803191891422978
BBOX Regression Loss:  0.0003497295943951165
Classification Loss:  0.725289570843732
BBOX Regression Loss:  0.0003531986054171015
Classification Loss:  0.7251392894321016
BBOX Regression Loss:  0.00036093782357595594
Classification Loss:  0.7250042291335119
BBOX Regression Loss:  0.00035470783348298733
Classification Loss:  0.7248723294999865
BBOX Regression Loss:  0.0003493155463150254
Classification Loss:  0.724734210261592
BBOX Regression Loss:  0.00034881044972549986
Classification Loss:  0.7246080139536917
BBOX Regression Loss:  0.00034898856467513185
Classification Loss:  0.7244952560101868
BBOX Regression Loss:  0.0003439144304677568
Classific

BBOX Regression Loss:  0.00032529832972551145
Classification Loss:  0.7201811436114203
BBOX Regression Loss:  0.00032532736237055916
Classification Loss:  0.7201510739007083
BBOX Regression Loss:  0.0003253330815706918
Classification Loss:  0.7201211816293223
BBOX Regression Loss:  0.0003262064147332431
Classification Loss:  0.7200914593390476
BBOX Regression Loss:  0.0003270352741475094
Classification Loss:  0.7200618628829252
BBOX Regression Loss:  0.0003277186252238709
Classification Loss:  0.7200324189726007
BBOX Regression Loss:  0.0003282310365529552
Classification Loss:  0.7200031314768508
BBOX Regression Loss:  0.0003283331782246629
Classification Loss:  0.7199739983876545
BBOX Regression Loss:  0.00032669473784389323
Classification Loss:  0.7170451058281792
BBOX Regression Loss:  0.00032736905678002924
Classification Loss:  0.717022270626492
BBOX Regression Loss:  0.00042809949796876793
Classification Loss:  0.7169985453287759
BBOX Regression Loss:  0.0004579451849201211
Class

BBOX Regression Loss:  0.0005044014374111229
Classification Loss:  0.7147496427043101
BBOX Regression Loss:  0.0005050686646038245
Classification Loss:  0.7147174851688338
BBOX Regression Loss:  0.0005062226133665928
Classification Loss:  0.7146851223272961
BBOX Regression Loss:  0.0005082408982687503
Classification Loss:  0.7146525167228711
BBOX Regression Loss:  0.0005099566505382921
Classification Loss:  0.7146196967717046
BBOX Regression Loss:  0.0005144084765964061
Classification Loss:  0.7145866845704326
BBOX Regression Loss:  0.0005181209455819739
Classification Loss:  0.7145534902055826
BBOX Regression Loss:  0.0005194422034733571
Classification Loss:  0.7145201178244602
BBOX Regression Loss:  0.0005203360185644448
Classification Loss:  0.7144865069511329
BBOX Regression Loss:  0.00052101140569414
Classification Loss:  0.7144526209870884
BBOX Regression Loss:  0.0005219031560481754
Classification Loss:  0.7144184863928592
BBOX Regression Loss:  0.000526285176579323
Classificati

BBOX Regression Loss:  0.0012985068009374967
Classification Loss:  0.7047031927835967
BBOX Regression Loss:  0.001307050054965093
Classification Loss:  0.704587111526476
BBOX Regression Loss:  0.0013254252710828073
Classification Loss:  0.7044697225220715
BBOX Regression Loss:  0.0013353677203107363
Classification Loss:  0.7043505768984656
BBOX Regression Loss:  0.001342685282217975
Classification Loss:  0.7042296852908728
BBOX Regression Loss:  0.0013509215575493413
Classification Loss:  0.7041068210523315
BBOX Regression Loss:  0.0013933532632884333
Classification Loss:  0.7039833331234241
BBOX Regression Loss:  0.001412204752905645
Classification Loss:  0.7038585653627552
BBOX Regression Loss:  0.0014221601079880885
Classification Loss:  0.7037315362575565
BBOX Regression Loss:  0.0014303650677650343
Classification Loss:  0.7036025722163975
BBOX Regression Loss:  0.0014449055211121112
Classification Loss:  0.703471804167095
BBOX Regression Loss:  0.0014670485843340923
Classification

BBOX Regression Loss:  0.01221907343631299
Classification Loss:  0.6558050054844801
BBOX Regression Loss:  0.01243993483487116
Classification Loss:  0.6551544421807511
BBOX Regression Loss:  0.01257881886100748
Classification Loss:  0.6545000331111686
BBOX Regression Loss:  0.012689733883783968
Classification Loss:  0.6538387409949874
BBOX Regression Loss:  0.012794095590312762
Classification Loss:  0.6531680113688099
BBOX Regression Loss:  0.013070687921238026
Classification Loss:  0.6524884210540264
BBOX Regression Loss:  0.013451857899350151
Classification Loss:  0.6518150921221133
BBOX Regression Loss:  0.013624103134168567
Classification Loss:  0.6511443715962677
BBOX Regression Loss:  0.013754016528281655
Classification Loss:  0.6504683117415189
BBOX Regression Loss:  0.013871810381787066
Classification Loss:  0.6497829765627234
BBOX Regression Loss:  0.013991612111384742
Classification Loss:  0.6490872867633372
BBOX Regression Loss:  0.014145983682002553
Classification Loss:  0.

## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch